# Reading the eso Output File

## 1. Introduction

The .eso output file of an EnergyPlus simulation contains the main results of the simulation such as the hourly air temperatures or monthly energy consumption. 
The .eso file is a text file in a non-standard format.

Some, but not all, of the contents of a .eso file can be viewed as a .csv file by including the `--readvars` argument when running an EnergyPlus simulation.
This runs an additional programme after the main simulation which converts some contents in the .eso file to a .csv file. 

`eprun` provides the :py:class:`~eprun.epeso.EPEso` class to directly read the .eso file and avoid the need for the `--readvars` step.


## 2. Accessing an EPEso object instance

In `eprun` the contents of the .eso file are viewed using the :py:class:`~eprun.epeso.EPEso` class. 
A :py:class:`~eprun.epeso.EPEso` object instance is created from the results of an EnergyPlus simulation run using the :py:meth:`~eprun.eprun.eprun` function.

This tutorial is based on the '1ZoneUncontrolled.idf' EnergyPlus input file which is provided in the 'ExampleFiles' directory of the EnergyPlus install directory.
The simulation is run using the 'USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw' EnergyPlus weather file.


In [1]:
from eprun import eprun
result=eprun(ep_dir='C:\EnergyPlusV9-4-0',
             idf_filepath='1ZoneUncontrolled.idf',
             epw_filepath='USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw',
             sim_dir='my_results')
eso=result.get_eso()
print(type(eso))

<class 'eprun.epeso.EPEso'>


## 3. Understanding EPEso simulation environments

An EnergyPlus input file can request that a simulation provide results for different 'simulation environments', 
such as a winter design day, a summer design day and a annual run period of hourly data.
The results of these simulation environments are stored in the same .eso file.

In our example, the '1ZoneUncontrolled.idf' contains the instructions for three simulation environments.
These are accessed using the :py:meth:`~eprun.epeso.EPEso.get_environments` method which returns a list of 
:py:class:`~eprun.epeso_simulation_environment.EPEsoSimulationEnviroment` object instances.

In [2]:
envs=eso.get_environments()
print(envs)

[EPEsoSimuationEnvironment("DENVER CENTENNIAL  GOLDEN   N ANN HTG 99% CONDNS DB"), EPEsoSimuationEnvironment("DENVER CENTENNIAL  GOLDEN   N ANN CLG 1% CONDNS DB=>MWB"), EPEsoSimuationEnvironment("RUN PERIOD 1")]


In [3]:
rp1=envs[2]
print(rp1)

EPEsoSimuationEnvironment("RUN PERIOD 1")


In [9]:
from pprint import pprint
interval_variables=rp1.get_interval_variables()
pprint(interval_variables)

(EPEsoIntervalVariable(sim_env="RUN PERIOD 1", report_code=7),
 EPEsoIntervalVariable(sim_env="RUN PERIOD 1", report_code=47),
 EPEsoIntervalVariable(sim_env="RUN PERIOD 1", report_code=74),
 EPEsoIntervalVariable(sim_env="RUN PERIOD 1", report_code=75),
 EPEsoIntervalVariable(sim_env="RUN PERIOD 1", report_code=76),
 EPEsoIntervalVariable(sim_env="RUN PERIOD 1", report_code=77))


In [15]:
interval_variable_7=interval_variables[0]
print(interval_variable_7.object_name)
print(interval_variable_7.quantity)
print(interval_variable_7.unit)
print(interval_variable_7.values[0:20])

Environment
Site Outdoor Air Drybulb Temperature
C
(7.0125, 7.2, 6.8875, 6.324999999999999, 5.0375, 4.4, 5.4624999999999995, 5.4125, 6.750000000000001, 8.487499999999999, 9.2125, 9.775, 10.375, 10.9125, 12.85, 13.9, 12.15, 11.1, 10.7875, 10.6)
